In [4]:
from rdkit import Chem
from rdkit.Chem import AllChem
from ase.calculators.mopac import MOPAC
from ase.io import read
import pandas as pd
import numpy as np
import time
from ase import Atoms

In [5]:
import mordred
from mordred import Calculator, descriptors
from rdkit.Chem import rdMolDescriptors

In [6]:
import json

In [7]:
# load periodic table json file
a=json.load(open("perotable.json"))

In [8]:
# create dictionary of elements...key: atomic number, value: atomic symbol
table=a['elements']
elemnt_table={}
# first 40 elements (not likely to find metals, etc), can change if wanted
for i in table[:40]:
    elemnt_table.update({i['number']:i['symbol']})
print(elemnt_table)

{1: 'H', 2: 'He', 3: 'Li', 4: 'Be', 5: 'B', 6: 'C', 7: 'N', 8: 'O', 9: 'F', 10: 'Ne', 11: 'Na', 12: 'Mg', 13: 'Al', 14: 'Si', 15: 'P', 16: 'S', 17: 'Cl', 18: 'Ar', 19: 'K', 20: 'Ca', 21: 'Sc', 22: 'Ti', 23: 'V', 24: 'Cr', 25: 'Mn', 26: 'Fe', 27: 'Co', 28: 'Ni', 29: 'Cu', 30: 'Zn', 31: 'Ga', 32: 'Ge', 33: 'As', 34: 'Se', 35: 'Br', 36: 'Kr', 37: 'Rb', 38: 'Sr', 39: 'Y', 40: 'Zr'}


In [9]:
# create a dictionary of energies per atom...key: atomic number, value: energy via PM7/MOPAC
energy_dict={}
for i,symbol in elemnt_table.items():
    
        #create ASE atoms object
        #set position to axis
        mol = Atoms(symbol, positions=[(0, 0, 0)])
        
        #calculate 
        mol.calc = MOPAC(label=f'atomic_energy{symbol}', task = 'UHF')

        #get potential energy
        pse = mol.get_potential_energy()

        energy_dict.update ({i:pse})
print(energy_dict)


{1: -11.07011, 2: -54.09664, 3: -4.80412, 4: -25.26494, 5: -50.84441, 6: -115.15732, 7: -177.85963, 8: -289.97952, 9: -462.69609, 10: -272.22972, 11: -5.81548, 12: -18.66074, 13: -62.20211, 14: -96.94941, 15: -170.60826, 16: -173.44087, 17: -265.24998, 18: -264.22578, 19: -4.88807, 20: -19.09281, 21: -37.25729, 22: -73.75413, 23: -119.28588, 24: -260.39534, 25: -226.56671, 26: -390.66498, 27: -398.01083, 28: -375.58231, 29: -673.16583, 30: -26.97859, 31: -57.28036, 32: -84.20003, 33: -123.88732, 34: -156.69076, 35: -221.41628, 36: -259.00171, 37: -4.12096, 38: -14.89116, 39: -34.67746, 40: -52.67139}


In [10]:
# look up atoms by their atomic number in the energy dictionary
# pull energies of those atoms and sum them for total energy
# this is the energy when atoms are infinitely far apart

In [33]:
# f = lambda x: x.GetAtomicNum()
f = lambda x: x.GetAtomicNum()

In [37]:
def atoms_to_energy(rdkit_atoms,energy_dict):
    sum_e=0
    atoms = rdkit_atoms.GetAtoms()
    for i in atoms:
        sum_e += energy_dict[f(i)]
    return sum_e

In [58]:
def get_diss_energy(smiles,energy_dict):
    """
    return Dissociate energy, ground state energy, atomic energy [float:float,float]
    """
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)
    # how do you get atoms
    inf_e=atoms_to_energy(mol,energy_dict)

    #embed molecule
    AllChem.EmbedMolecule(mol)

    #create xyz file
    fileName = 'xyz/'+"tester" + '.xyz'
    Chem.MolToXYZFile(mol,fileName)

    #use ASE to read xyz file
    mol = read(fileName)
    mol.calc = MOPAC(label=f'GS_energy', task = 'UHF')
    gs_e=mol.get_potential_energy()
    # mol.get_chemical_symbols()
    #use ASE to read xyz file
    DE = gs_e - inf_e
    return (DE,gs_e,inf_e)


In [59]:
dataset = pd.read_csv('../../Data/Solubility/dataset-H.csv')

In [60]:
#SMILES
smiles = dataset.columns[4]
smiles_list = np.array(dataset[smiles])
# print (smiles_list)

#ID
ID = dataset.columns[0]
id_list = np.array(dataset[ID])
# print (id_list)

#initialize lists
failures = []
pse_list = []
ae_list = []

In [65]:
def generate():
    
    start = time.time()
    print ('starting...')
    for i,smiles in enumerate(smiles_list):
#         try:
    
            DE,pse,inf_e=get_diss_energy(smiles,energy_dict)

            #save into list
            pse_list.append(pse)
            
            # atomistic energy to list
            ae_list.append(DE)
            print (f"{id_list[i]} Done ae = :{DE}")
            ###########################################
            
#         except Exception as e :
#             print (f'error in{id_list[i]} reason:{e}')
            
#             pse_list.append(None)
#             ae_list.append(None)

        
            
    print ('DONE. time taken = ', time.time()- start)

In [66]:
generate()

starting...
H-1 Done ae = :-110.12613999999917
H-2 Done ae = :-41.87789000000009
H-3 Done ae = :-30.593290000000025
H-4 Done ae = :-85.93446999999901
H-5 Done ae = :-37.04395000000011
H-6 Done ae = :-37.04575
H-7 Done ae = :-48.61833999999999
H-8 Done ae = :-37.424409999999966


KeyboardInterrupt: 

In [211]:
# # bond info
# m = Chem.MolFromSmiles('C1OC1')
# for atom in m.GetAtoms():
#     print(m.GetBonds()[0].GetBondType())

In [ ]:
# get type of each atom present in the SMILES string
# get number of each atom type present in SMILES string

In [24]:
# calculate the energy via MOPAC/PM7 for each individual atom present in the SMILES string

# multiply the energy of each atom type by its number in the molecule

# sum the products for the "upper bound" energy (atoms inf far apart)

# subtract the PM7 ground state energy for the molecule from the sum of the products (inf far apart sum - PM7 GS) to get atomistic energy (delta E)

# use this as the energy value to featurize

100%|██████████| 578/578 [00:04<00:00, 132.37it/s]
